In [5]:
import json
import websocket
import datetime
import numpy as np
import pandas as pd
from datetime import datetime, timezone
import time
from Thalassa_Regime_Classifier.preprocessing_streamed_data import preprocessing_streamed_data

ModuleNotFoundError: No module named 'Thalassa_Regime_Classifier'

In [23]:
print(time.mktime(datetime.now().timetuple()))
print(int(time.mktime(datetime.now().timetuple())))

1653586152.0
1653586152


In [2]:
def my_json(json_message):
    size = len(np.array(json_message['bids'])[:,0])    
    return {
        **{'ts':[int(time.mktime(datetime.now().timetuple()))]},
        **{'bp'+str(key):[float(value)] for key,value in zip(np.arange(0,size)+1,np.array(json_message['bids'])[:,0])},
        **{'bs'+str(key):[float(value)] for key,value in zip(np.arange(0,size)+1,np.array(json_message['bids'])[:,1])},
        **{'ap'+str(key):[float(value)] for key,value in zip(np.arange(0,size)+1,np.array(json_message['asks'])[:,0])},
        **{'as'+str(key):[float(value)] for key,value in zip(np.arange(0,size)+1,np.array(json_message['asks'])[:,1])}}
    
def my_json_0(size):    
    return {
        **{'ts':[int(time.mktime(datetime.now().timetuple()))]},
        **{'bp'+str(key):[value] for key,value in zip(np.arange(0,size)+1,(np.arange(0,size)+1)*np.nan)},
        **{'bs'+str(key):[value] for key,value in zip(np.arange(0,size)+1,(np.arange(0,size)+1)*np.nan)},
        **{'ap'+str(key):[value] for key,value in zip(np.arange(0,size)+1,(np.arange(0,size)+1)*np.nan)},
        **{'as'+str(key):[value] for key,value in zip(np.arange(0,size)+1,(np.arange(0,size)+1)*np.nan)}}

In [107]:

df_0 = pd.DataFrame.from_dict(my_json_0(5))

def ws_trades(df_0): 
    symbol = 'BTCUSDT'
    depth = 5
    socket = 'wss://stream.binance.com:9443/ws/{}@depth{}'.format(symbol.lower(),
                                                                depth)
    
    
    def on_message(wsapp,message):  
        json_message = json.loads(message)
        handle_trades(json_message)

    def on_error(wsapp,error):
        print(error)

    wsapp = websocket.WebSocketApp(socket, on_message=on_message, on_error=on_error)
    wsapp.run_forever()
    
def handle_trades(json_message):    
    df = pd.DataFrame.from_dict(my_json(json_message))
    df_0 = df_0.append(df, ignore_index = True)
    
    print(df_0)
    
    
    
    print("-----------------------")

In [3]:
class StreamingData():
    def __init__(self,df_0):
        self.df_0=df_0
        self.symbol = 'BTCUSDT'
        self.depth = 5        
        self.socket = 'wss://stream.binance.com:9443/ws/{}@depth{}'.format(self.symbol.lower(),self.depth)
        
    def handle_trades(self,json_message):    
        df = pd.DataFrame.from_dict(my_json(json_message))
        self.df_0 = self.df_0.append(df, ignore_index = True)       
        print(self.df_0)
        print(self.preprocessing_streamed_data(self.df_0, rolling_window=2))
                
    def my_return(self):
        return self.df_0

    def on_message(self,wsapp,message):  
        json_message = json.loads(message)
        self.handle_trades(json_message)

    def on_error(self,wsapp,error):
        print(error)

    def start(self):
        wsapp = websocket.WebSocketApp(self.socket, on_message=self.on_message, on_error=self.on_error)
        wsapp.run_forever()
        
    def preprocessing_streamed_data(self,df_ob, rolling_window=2):
        '''preprocessing of data for streamed data'''
        
        print('hola1')
        # from unix timestamp to human date
        unix_timestamp = lambda x: datetime.fromtimestamp(x/1000.0, tz=timezone.utc).strftime('%Y-%m-%d %H:%M:%S')
        str2date = lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S')
        
        print('hola2')
        df_ob['ts']=df_ob['ts'].apply(unix_timestamp).apply(str2date)
        
        # aggregating by seconds     
        df_agg = df_ob.groupby(pd.Grouper(key='ts', axis=0, freq='S')).mean()
        # applying rolling window of rolling_window lenght
        df_agg = df_agg.rolling(str(rolling_window)+'S').mean()
        df_agg.reset_index(inplace=True)
        print(df_agg.head())
        # dropping useless columns
        #df_agg.drop(columns=['ts'], inplace=True)

        return df_agg

    

In [4]:
df_0 = pd.DataFrame.from_dict(my_json_0(5))
w = StreamingData(df_0)
w.start()

# w.my_return()

/tmp/ipykernel_18158/1311226267.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df_0 = self.df_0.append(df, ignore_index = True)


           ts       bp1       bp2       bp3       bp4       bp5      bs1  \
0  1653586676       NaN       NaN       NaN       NaN       NaN      NaN   
1  1653586677  29609.99  29609.97  29608.44  29607.14  29607.13  0.05081   

       bs2      bs3      bs4  ...      ap1       ap2       ap3       ap4  \
0      NaN      NaN      NaN  ...      NaN       NaN       NaN       NaN   
1  0.01581  0.00067  0.07483  ...  29610.0  29610.01  29610.02  29610.04   

       ap5       as1      as2      as3      as4      as5  
0      NaN       NaN      NaN      NaN      NaN      NaN  
1  29610.1  14.63605  0.01295  0.01465  0.00846  0.02025  

[2 rows x 21 columns]
hola1
hola2
                   ts       bp1       bp2       bp3       bp4       bp5  \
0 1970-01-20 03:19:46  29609.99  29609.97  29608.44  29607.14  29607.13   

       bs1      bs2      bs3      bs4  ...      ap1       ap2       ap3  \
0  0.05081  0.01581  0.00067  0.07483  ...  29610.0  29610.01  29610.02   

        ap4      ap5       a

/tmp/ipykernel_18158/1311226267.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df_0 = self.df_0.append(df, ignore_index = True)


                    ts       bp1       bp2       bp3       bp4       bp5  \
0  1970-01-20 03:19:46       NaN       NaN       NaN       NaN       NaN   
1  1970-01-20 03:19:46  29609.99  29609.97  29608.44  29607.14  29607.13   
2           1653586678  29607.13  29607.12  29606.96  29606.95  29606.89   

       bs1      bs2      bs3      bs4  ...       ap1       ap2       ap3  \
0      NaN      NaN      NaN      NaN  ...       NaN       NaN       NaN   
1  0.05081  0.01581  0.00067  0.07483  ...  29610.00  29610.01  29610.02   
2  1.77227  0.00311  0.02395  0.00036  ...  29607.14  29607.24  29607.49   

        ap4       ap5       as1      as2      as3      as4      as5  
0       NaN       NaN       NaN      NaN      NaN      NaN      NaN  
1  29610.04  29610.10  14.63605  0.01295  0.01465  0.00846  0.02025  
2  29607.63  29608.45   2.32136  0.00075  0.90000  0.25000  0.30000  

[3 rows x 21 columns]
hola1
hola2
unsupported operand type(s) for /: 'Timestamp' and 'float'


/tmp/ipykernel_18158/1311226267.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df_0 = self.df_0.append(df, ignore_index = True)


                    ts       bp1       bp2       bp3       bp4       bp5  \
0  1970-01-20 03:19:46       NaN       NaN       NaN       NaN       NaN   
1  1970-01-20 03:19:46  29609.99  29609.97  29608.44  29607.14  29607.13   
2           1653586678  29607.13  29607.12  29606.96  29606.95  29606.89   
3           1653586679  29607.13  29607.12  29607.06  29606.96  29606.95   

       bs1      bs2      bs3      bs4  ...       ap1       ap2       ap3  \
0      NaN      NaN      NaN      NaN  ...       NaN       NaN       NaN   
1  0.05081  0.01581  0.00067  0.07483  ...  29610.00  29610.01  29610.02   
2  1.77227  0.00311  0.02395  0.00036  ...  29607.14  29607.24  29607.49   
3  2.11487  0.00311  0.08092  0.02395  ...  29607.14  29607.18  29607.24   

        ap4       ap5       as1      as2      as3      as4      as5  
0       NaN       NaN       NaN      NaN      NaN      NaN      NaN  
1  29610.04  29610.10  14.63605  0.01295  0.01465  0.00846  0.02025  
2  29607.63  29608.45   2.32

In [98]:
ws_trades(df_0)

local variable 'df_0' referenced before assignment
local variable 'df_0' referenced before assignment
local variable 'df_0' referenced before assignment
local variable 'df_0' referenced before assignment



In [55]:
  
# initialising dictionaries
ini_dictionary1 = {'nikhil': 1, 'akash' : 5,
                     'manjeet' : 10, 'akshat' : 15}
ini_dictionary2 = {'akash' : 7, 'akshat' : 5,
                                          'm' : 15}

ini_dictionary1.update(ini_dictionary2)
ini_dictionary1  


{'nikhil': 1, 'akash': 7, 'manjeet': 10, 'akshat': 5, 'm': 15}

In [14]:
json_message ={'lastUpdateId': 19493390097, 
               'bids': [['29724.70000000', '8.90412000'], ['29747.650000', '0.00429000'], ['29724.62000000', '0.08293000'], ['29724.61000000', '0.01998000'], ['29724.57000000', '0.16835000']], 
               'asks': [['29724.71000000', '0.00001000'], ['29724.72000000', '0.00075000'], ['29724.73000000', '0.01237000'], ['29724.75000000', '0.01237000'], ['29724.77000000', '0.01000000']]}


{'bp'+str(key):value for key,value in zip(np.arange(0,5),np.array(json_message['bids'])[:,0]) }
{'bs'+str(key):value for key,value in zip(np.arange(0,5),np.array(json_message['bids'])[:,1]) }
{'bp'+str(key):value for key,value in zip(np.arange(0,5),np.array(json_message['bids'])[:,0]) }
{'bp'+str(key):value for key,value in zip(np.arange(0,5),np.array(json_message['bids'])[:,0]) }


{'bp0': '29724.70000000',
 'bp1': '29747.650000',
 'bp2': '29724.62000000',
 'bp3': '29724.61000000',
 'bp4': '29724.57000000'}

In [ ]:
json_message ={'lastUpdateId': 19493390097, 
               'bids': [['29724.70000000', '8.90412000'], ['nan', '0.00429000'], ['29724.62000000', '0.08293000'], ['29724.61000000', '0.01998000'], ['29724.57000000', '0.16835000']], 
               'asks': [['29724.71000000', '0.00001000'], ['29724.72000000', '0.00075000'], ['29724.73000000', '0.01237000'], ['29724.75000000', '0.01237000'], ['29724.77000000', '0.01000000']]}
aaa(bp1) bid price 1 = 29724.70000000
(bs1) bid size 1 = 8.90412000

(ap1) asd price 1 = 29724.71000000
(as1) ask size 1 = 0000100

{'ts':json_message['lastUpdateId'],
  'bp1':json_message['bids'][0][0],
  'bp1':json_message['bids'][0][0],
  'bp1':json_message['bids'][0][0],
  'bp1':json_message['bids'][0][0],
  'bp1':json_message['bids'][0][0],
  'bp1':json_message['bids'][0][0],
  'bp1':json_message['bids'][0][0],
  'bp1':json_message['bids'][0][0],
  'bp1':json_message['bids'][0][0],
  'bp1':json_message['bids'][0][0],  
  'bp1':json_message['bids'][0][0],
  }

symbol,ts,last_update_id,bp1,bs1,bp2,bs2,bp3,bs3,bp4,bs4,bp5,bs5,bp6,bs6,bp7,bs7,bp8,bs8,bp9,bs9,bp10,bs10,bp11,bs11,bp12,bs12,bp13,bs13,bp14,bs14,bp15,bs15,bp16,bs16,bp17,bs17,bp18,bs18,bp19,bs19,bp20,bs20,ap1,as1,ap2,as2,ap3,as3,ap4,as4,ap5,as5,ap6,as6,ap7,as7,ap8,as8,ap9,as9,ap10,as10,ap11,as11,ap12,as12,ap13,as13,ap14,as14,ap15,as15,ap16,as16,ap17,as17,ap18,as18,ap19,as19,ap20,as20

In [ ]:
import pyspark
from pyspark.streaming import StreamingContext

In [ ]:
#Uncomment the next line to run the code block on jupyter. Keep it commented if copy-pasting into the pyspark shell
sc = pyspark.SparkContext()

# This tells Spark Streaming to bacth-up the contents of a data stream and "ingest" them every 10 seconds.
ssc = StreamingContext(sc,10)

# Tell spark to listen on port 9999 of our localhost.
lines = ssc.socketTextStream('wss://stream.binance.com/ws/BTCUSDT@depth5', 9443)
lines = ssc.socketTextStream('localhost', 9999)    

words = lines.flatMap(lambda line : line.split(" "))

pairs = words.map(lambda word: (word, 1))
wordCount = pairs.reduceByKey(lambda a, b: a + b)

wordCount.pprint()

ssc.start()
ssc.awaitTermination()

In [5]:
#Uncomment the next line to run the code block on jupyter. Keep it commented if copy-pasting into the pyspark shell
sc = pyspark.SparkContext()

# This tells Spark Streaming to bacth-up the contents of a data stream and "ingest" them every 10 seconds.
ssc = StreamingContext(sc,10)

# Tell spark to listen on port 9999 of our localhost.
lines = ssc.socketTextStream('wss://stream.binance.com/ws/BTCUSDT@depth5', 9443)



lines.pprint()

ssc.start()
ssc.awaitTermination()

22/05/25 13:09:12 ERROR ReceiverTracker: Deregistered receiver for stream 0: Error starting receiver 0 - java.net.UnknownHostException: wss://stream.binance.com/ws/BTCUSDT@depth5
	at java.net.AbstractPlainSocketImpl.connect(AbstractPlainSocketImpl.java:184)
	at java.net.SocksSocketImpl.connect(SocksSocketImpl.java:392)
	at java.net.Socket.connect(Socket.java:607)
	at java.net.Socket.connect(Socket.java:556)
	at java.net.Socket.<init>(Socket.java:452)
	at java.net.Socket.<init>(Socket.java:229)
	at org.apache.spark.streaming.dstream.SocketReceiver.onStart(SocketInputDStream.scala:61)
	at org.apache.spark.streaming.receiver.ReceiverSupervisor.startReceiver(ReceiverSupervisor.scala:149)
	at org.apache.spark.streaming.receiver.ReceiverSupervisor.start(ReceiverSupervisor.scala:131)
	at org.apache.spark.streaming.scheduler.ReceiverTracker$ReceiverTrackerEndpoint.$anonfun$startReceiver$1(ReceiverTracker.scala:596)
	at org.apache.spark.streaming.scheduler.ReceiverTracker$ReceiverTrackerEndpoin

-------------------------------------------
Time: 2022-05-25 13:09:20
-------------------------------------------



22/05/25 13:09:20 ERROR ReceiverSupervisorImpl: Stopped receiver with error: java.net.UnknownHostException: wss://stream.binance.com/ws/BTCUSDT@depth5
22/05/25 13:09:20 ERROR Executor: Exception in task 0.0 in stage 19.0 (TID 19)
java.net.UnknownHostException: wss://stream.binance.com/ws/BTCUSDT@depth5
	at java.net.AbstractPlainSocketImpl.connect(AbstractPlainSocketImpl.java:184)
	at java.net.SocksSocketImpl.connect(SocksSocketImpl.java:392)
	at java.net.Socket.connect(Socket.java:607)
	at java.net.Socket.connect(Socket.java:556)
	at java.net.Socket.<init>(Socket.java:452)
	at java.net.Socket.<init>(Socket.java:229)
	at org.apache.spark.streaming.dstream.SocketReceiver.onStart(SocketInputDStream.scala:61)
	at org.apache.spark.streaming.receiver.ReceiverSupervisor.startReceiver(ReceiverSupervisor.scala:149)
	at org.apache.spark.streaming.receiver.ReceiverSupervisor.start(ReceiverSupervisor.scala:131)
	at org.apache.spark.streaming.scheduler.ReceiverTracker$ReceiverTrackerEndpoint.$anonf

Py4JError: An error occurred while calling o421.awaitTermination

22/05/25 13:09:28 ERROR ReceiverSupervisorImpl: Stopped receiver with error: java.net.UnknownHostException: wss://stream.binance.com/ws/BTCUSDT@depth5
22/05/25 13:09:28 ERROR Executor: Exception in task 0.0 in stage 41.0 (TID 41)
java.net.UnknownHostException: wss://stream.binance.com/ws/BTCUSDT@depth5
	at java.net.AbstractPlainSocketImpl.connect(AbstractPlainSocketImpl.java:184)
	at java.net.SocksSocketImpl.connect(SocksSocketImpl.java:392)
	at java.net.Socket.connect(Socket.java:607)
	at java.net.Socket.connect(Socket.java:556)
	at java.net.Socket.<init>(Socket.java:452)
	at java.net.Socket.<init>(Socket.java:229)
	at org.apache.spark.streaming.dstream.SocketReceiver.onStart(SocketInputDStream.scala:61)
	at org.apache.spark.streaming.receiver.ReceiverSupervisor.startReceiver(ReceiverSupervisor.scala:149)
	at org.apache.spark.streaming.receiver.ReceiverSupervisor.start(ReceiverSupervisor.scala:131)
	at org.apache.spark.streaming.scheduler.ReceiverTracker$ReceiverTrackerEndpoint.$anonf

-------------------------------------------
Time: 2022-05-25 13:09:30
-------------------------------------------



22/05/25 13:09:30 ERROR ReceiverSupervisorImpl: Stopped receiver with error: java.net.UnknownHostException: wss://stream.binance.com/ws/BTCUSDT@depth5
22/05/25 13:09:30 ERROR Executor: Exception in task 0.0 in stage 44.0 (TID 44)
java.net.UnknownHostException: wss://stream.binance.com/ws/BTCUSDT@depth5
	at java.net.AbstractPlainSocketImpl.connect(AbstractPlainSocketImpl.java:184)
	at java.net.SocksSocketImpl.connect(SocksSocketImpl.java:392)
	at java.net.Socket.connect(Socket.java:607)
	at java.net.Socket.connect(Socket.java:556)
	at java.net.Socket.<init>(Socket.java:452)
	at java.net.Socket.<init>(Socket.java:229)
	at org.apache.spark.streaming.dstream.SocketReceiver.onStart(SocketInputDStream.scala:61)
	at org.apache.spark.streaming.receiver.ReceiverSupervisor.startReceiver(ReceiverSupervisor.scala:149)
	at org.apache.spark.streaming.receiver.ReceiverSupervisor.start(ReceiverSupervisor.scala:131)
	at org.apache.spark.streaming.scheduler.ReceiverTracker$ReceiverTrackerEndpoint.$anonf

In [6]:
ssc.stop()

22/05/25 13:09:40 ERROR ReceiverSupervisorImpl: Stopped receiver with error: java.net.UnknownHostException: wss://stream.binance.com/ws/BTCUSDT@depth5
22/05/25 13:09:40 ERROR Executor: Exception in task 0.0 in stage 69.0 (TID 69)
java.net.UnknownHostException: wss://stream.binance.com/ws/BTCUSDT@depth5
	at java.net.AbstractPlainSocketImpl.connect(AbstractPlainSocketImpl.java:184)
	at java.net.SocksSocketImpl.connect(SocksSocketImpl.java:392)
	at java.net.Socket.connect(Socket.java:607)
	at java.net.Socket.connect(Socket.java:556)
	at java.net.Socket.<init>(Socket.java:452)
	at java.net.Socket.<init>(Socket.java:229)
	at org.apache.spark.streaming.dstream.SocketReceiver.onStart(SocketInputDStream.scala:61)
	at org.apache.spark.streaming.receiver.ReceiverSupervisor.startReceiver(ReceiverSupervisor.scala:149)
	at org.apache.spark.streaming.receiver.ReceiverSupervisor.start(ReceiverSupervisor.scala:131)
	at org.apache.spark.streaming.scheduler.ReceiverTracker$ReceiverTrackerEndpoint.$anonf

-------------------------------------------
Time: 2022-05-25 13:09:40
-------------------------------------------



22/05/25 13:09:50 ERROR JobScheduler: Error in job generator
java.lang.NullPointerException
	at org.apache.spark.storage.BlockManagerMaster.removeRdd(BlockManagerMaster.scala:162)
	at org.apache.spark.SparkContext.unpersistRDD(SparkContext.scala:1912)
	at org.apache.spark.rdd.RDD.unpersist(RDD.scala:222)
	at org.apache.spark.streaming.dstream.DStream.$anonfun$clearMetadata$6(DStream.scala:459)
	at org.apache.spark.streaming.dstream.DStream.$anonfun$clearMetadata$6$adapted(DStream.scala:458)
	at scala.collection.mutable.HashMap$$anon$2.$anonfun$foreach$3(HashMap.scala:158)
	at scala.collection.mutable.HashTable.foreachEntry(HashTable.scala:237)
	at scala.collection.mutable.HashTable.foreachEntry$(HashTable.scala:230)
	at scala.collection.mutable.HashMap.foreachEntry(HashMap.scala:44)
	at scala.collection.mutable.HashMap$$anon$2.foreach(HashMap.scala:158)
	at org.apache.spark.streaming.dstream.DStream.clearMetadata(DStream.scala:458)
	at org.apache.spark.streaming.dstream.DStream.$anonfu